<a href="https://colab.research.google.com/github/KayvanShah1/Eye-and-Face-Direction-Detection/blob/master/pipeline_testing/Data_Pipeline_Testing-20210725-2227.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dependencies

## Packages and Repositories

In [1]:
# @title ##Installation
!pip install ipython-autotime
!pip install plotly
!pip install category_encoders
!pip install sentence-transformers
!pip install pgeocode
!pip install geopandas
!pip install python-decouple
!pip install geopy==1.22.0

%load_ext autotime

time: 3.98 ms (started: 2021-07-25 15:32:45 +00:00)


In [2]:
#@title ###Run to import **PPP Scrapers** repo { form-width: "30%" }
import os
import shutil
from getpass import getpass

# username = input('GitLab username: ')
# password = getpass('GitLab password: ')
os.environ['GITLAB_AUTH'] = "kayvanshah1" + ':' + "kayvan29Shah"

os.chdir('/content/')
!git clone https://$GITLAB_AUTH@gitlab.com/atlas_wizard/intellisearch/pppscrapers.git --branch master --single-branch
os.chdir('/content/pppscrapers')
!python setup.py sdist bdist_wheel
os.chdir('/content')
!pip3 uninstall /content/pppscrapers/dist/ppp_scrapers-1.0.0-py2.py3-none-any.whl -y
!pip3 install /content/pppscrapers/dist/ppp_scrapers-1.0.0-py2.py3-none-any.whl
os.chdir('/content/pppscrapers')

fatal: destination path 'pppscrapers' already exists and is not an empty directory.
running sdist
running egg_info
writing ppp_scrapers.egg-info/PKG-INFO
writing dependency_links to ppp_scrapers.egg-info/dependency_links.txt
writing top-level names to ppp_scrapers.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'ppp_scrapers.egg-info/SOURCES.txt'

running check


creating ppp_scrapers-1.0.0
creating ppp_scrapers-1.0.0/docs
creating ppp_scrapers-1.0.0/ppp_scrapers.egg-info
creating ppp_scrapers-1.0.0/prefect_flows
creating ppp_scrapers-1.0.0/src
creating ppp_scrapers-1.0.0/src/ppp_scrapers
creating ppp_scrapers-1.0.0/src/ppp_scrapers/data_processing
creating ppp_scrapers-1.0.0/src/ppp_scrapers/data_processing/clean
creating ppp_scrapers-1.0.0/src/ppp_scrapers/data_processing/embeddings
creating ppp_scrapers-1.0.0/src/ppp_scrapers/data_processing/geocode
creating ppp_scrapers-1.0.0/src/ppp_scrapers/data_processing/standardization
creating ppp_scrapers-1.0.0/src

## Imports

In [3]:
# @title ###Additional Imports
import pandas as pd
import geopandas as gpd
import numpy as np
import os,re
from datetime import datetime
import pickle
import pgeocode

# from google.colab import auth
# import gspread
# import gspread_dataframe as gsd
# from oauth2client.client import GoogleCredentials

# auth.authenticate_user()
# gc = gspread.authorize(GoogleCredentials.get_application_default())

import plotly.express as px
import plotly.graph_objects as go 
from tqdm.notebook import tqdm

import hashlib

time: 695 ms (started: 2021-07-25 15:32:51 +00:00)


In [4]:
# @title ### GCS Bucket
from ppp_scrapers.utils import load_config_yaml
from ppp_scrapers.utils.bucket import (
    connect_to_buffer_bucket,
    push_csv_to_buffer_bucket, 
    read_csv_from_buffer_bucket,  
    read_json_from_buffer_bucket,
    push_json_to_buffer_bucket,
    read_pickle_from_buffer_bucket,
    push_pickle_to_buffer_bucket
)

config = load_config_yaml()
bucket = connect_to_buffer_bucket()

Successfully connected to the bucket "psd_data"
time: 354 ms (started: 2021-07-25 15:32:52 +00:00)


# Data Reliabilty Check

In [35]:
df = read_csv_from_buffer_bucket(bucket,'projects/samgovco/clean/year_wise/ContractOpportunitiesFullCSV.csv')
print(df.shape)
df.head()

(175181, 56)


,tender_id,tender_name,solicitation_id,department_or_industry_agency,cgac,sub_tier,fpds_code,office,aac_code,posted_date,type,basetype,archive_type,archive_date,set_a_side_code,set_a_side,response_deadline,naics_code,classification_code,pop_street_address,pop_city,pop_state,pop_zip,pop_country,active,award_number,award_date,award_usd,awardee,primary_contact_title,primary_contact_fullname,primary_contact_email,primary_contact_phone,primary_contact_fax,secondary_contact_title,secondary_contact_fullname,secondary_contact_email,secondary_contact_phone,secondary_contact_fax,organization_type,state,city,zipcode,countrycode,additional_info_link,url,description,country_code,country_name,region_name,region_code,investment_amt_usd,naics_name,project_or_tender,source,aug_id
0,f46824eb239941a7ae2e32f0171b0d4e,Healthy Aging In Neighborhoods Of Diversity Ac...,HHS-NIH-NIDA(AG)-SBSS-75N95021Q00251,"Health And Human Services, Department Of",75.0,National Institutes Of Health,7529,NATIONAL INSTITUTES OF HEALTH NIDA,75N950,2021-06-22,Sources Sought,Sources Sought,auto15,2021-07-21,NaN,NaN,2021-07-06,541611,R499,NaN,NaN,NaN,NaN,USA,Yes,NaN,NaN,NaN,NaN,NaN,Fred Ettehadieh,Fred.Ettehadieh@nih.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OFFICE,MD,Bethesda,20892,USA,NaN,https://beta.sam.gov/opp/f46824eb239941a7ae2e3...,This is a Small Business Sources Sought notice...,USA,U.S.,North America,NAC,NaN,Administrative Management and General Manageme...,T,SAMGOVCO,SAMGOVCO_f46824eb239941a7ae2e32f0171b0d4e
1,8c98677a2f8e4de48bc5759edba78f91,Home Living Essentials & Mattresses,140A2321Q0424,"Interior, Department Of The",14.0,Bureau Of Indian Affairs,1450,INDIAN EDUCATION ACQUISITION OFFICE,140A23,2021-06-22,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,auto15,2021-07-10,ISBEE,Indian Small Business Economic Enterprise (ISB...,2021-06-25,337125,7105,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,"Begay, Janice",Janice.Begay@bie.edu,7857498419,7858302760,NaN,NaN,NaN,NaN,NaN,OFFICE,NM,Albuquerque,87104,USA,NaN,https://beta.sam.gov/opp/8c98677a2f8e4de48bc57...,Quotes must be submitted to janice.begay@bie.e...,USA,U.S.,North America,NAC,NaN,Household Furniture (except Wood and Metal) Ma...,T,SAMGOVCO,SAMGOVCO_8c98677a2f8e4de48bc5759edba78f91
2,3340d5629a3d4c6c98eb8f4d3eefd9a4,Fan Coil,N0024421Q0082,Dept Of Defense,17.0,Dept Of The Navy,1700,NAVSUP FLT LOG CTR SAN DIEGO,N00244,2021-06-22,Solicitation,Solicitation,auto15,2021-07-09,NaN,NaN,2021-06-24,333415,4120,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Telephone: 6195565519,charles.e.fletcher1@navy.mil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OFFICE,CA,San Diego,92136,USA,NaN,https://beta.sam.gov/opp/3340d5629a3d4c6c98eb8...,EXTREMELY URGENT BRAND NAME OR EQUAL ALL OR NO...,USA,U.S.,North America,NAC,NaN,Air-Conditioning and Warm Air Heating Equipmen...,T,SAMGOVCO,SAMGOVCO_3340d5629a3d4c6c98eb8f4d3eefd9a4
3,a8cae1fa067f497581061dbe857cfd18,Abeyance Cryo Solutions A440 Ln2 Vapor Freezer,HHS-NIH-NIDA(AG)-SBSS-75N95021Q00250,"Health And Human Services, Department Of",75.0,National Institutes Of Health,7529,NATIONAL INSTITUTES OF HEALTH NIDA,75N950,2021-06-22,Sources Sought,Sources Sought,auto15,2021-07-21,NaN,NaN,2021-07-06,333415,6640,NaN,NaN,NaN,NaN,USA,Yes,NaN,NaN,NaN,NaN,NaN,Fred Ettehadieh,Fred.Ettehadieh@nih.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OFFICE,MD,Bethesda,20892,USA,NaN,https://beta.sam.gov/opp/a8cae1fa067f497581061...,This is a Small Business Sources Sought notice...,USA,U.S.,North America,NAC,NaN,Air-Conditioning and Warm Air Heating Equipmen...,T,SAMGOVCO,SAMGOVCO_a8cae1fa067f497581061dbe857cfd18
4,bbf6c6513caa44b3a19fd81d7a7bee2e,Quasonix Telemetry Transmitters And Quasonix R...,FA930221Q0035,Dept Of Defense,57.0,Dept Of The Air Force,5700,FA9302 AFTC PZIE,FA9302,2021-06-22,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,autocustom,2021-09-15,NaN,NaN,2021-06-30,334511,6930,NaN,Edwards,CA,93524,USA,Yes,NaN,NaN,NaN,NaN,NaN,Terence Vickers,terence.vickers@us.af.mil,6612772232,6612770470,NaN,Wendell Brown

time: 12 s (started: 2021-07-25 16:42:33 +00:00)


In [6]:
# df.country_code.value_counts(dropna=False)
# df[(~df['zipcode'].isnull())&(df['country_code']!='USA')]
# df[df['country_code']=='IND']
# df['region_name'].unique()
# df.info()

time: 3.02 ms (started: 2021-07-25 13:51:43 +00:00)


In [36]:
{
    f"{col}":f"{col.lower().replace(' ','_')}" for col in df.columns
}

{'aac_code': 'aac_code',
 'active': 'active',
 'additional_info_link': 'additional_info_link',
 'archive_date': 'archive_date',
 'archive_type': 'archive_type',
 'aug_id': 'aug_id',
 'award_date': 'award_date',
 'award_number': 'award_number',
 'award_usd': 'award_usd',
 'awardee': 'awardee',
 'basetype': 'basetype',
 'cgac': 'cgac',
 'city': 'city',
 'classification_code': 'classification_code',
 'country_code': 'country_code',
 'country_name': 'country_name',
 'countrycode': 'countrycode',
 'department_or_industry_agency': 'department_or_industry_agency',
 'description': 'description',
 'fpds_code': 'fpds_code',
 'investment_amt_usd': 'investment_amt_usd',
 'naics_code': 'naics_code',
 'naics_name': 'naics_name',
 'office': 'office',
 'organization_type': 'organization_type',
 'pop_city': 'pop_city',
 'pop_country': 'pop_country',
 'pop_state': 'pop_state',
 'pop_street_address': 'pop_street_address',
 'pop_zip': 'pop_zip',
 'posted_date': 'posted_date',
 'primary_contact_email': 'pr

time: 8.96 ms (started: 2021-07-25 16:42:56 +00:00)


# Working Playground

## Geocoding

In [ ]:
#@title ###Geocoding Script { form-width: "20%" }
import pandas as pd
import numpy as np
import warnings
import traceback

from ppp_scrapers.utils.geocode2 import Geocoder
from ppp_scrapers.utils import load_config_yaml
from ppp_scrapers.utils.bucket import (
    connect_to_buffer_bucket,
    push_csv_to_buffer_bucket,
    read_csv_from_buffer_bucket,
)


class WBPPIGeocoder:
    def __init__(self, **kwargs):
        self.config = kwargs.get("config")
        self.bucket = connect_to_buffer_bucket()
        self.geocoder = Geocoder()
        self.geodf: pd.core.frame.DataFrame = None

    def load_data(self):
        try:
            df = read_csv_from_buffer_bucket(
                self.bucket, self.config["cleaned_data_path"]
            )
            return df
        except FileNotFoundError as e:
            print(f"Unable to read file from bucket: {e}", traceback.format_exc())

    def geocode(self, df):
        df["location"] = df["location"].replace([np.nan],"")
        df["country"] = df["country"].replace([np.nan],"")
        df["location_col"] = df[["location","country"]].apply(
            lambda x: ", ".join(x.tolist()), axis=1
        )

        df = self.geocoder.geocode_df(
            df, "location_col"
        )
        unindentified_df = df[df["map_coordinates"].isnull()]
        unindentified_df = self.geocoder.geocode_df(
            unindentified_df, "country"
        )
        df = df[~df["map_coordinates"].isnull()]
        df = df.append(unindentified_df, ignore_index=True)
        df.drop(columns=["location_col"], inplace=True)
        return df

    def save_data(self, df, save_path):
        try:
            push_csv_to_buffer_bucket(self.bucket, df, save_path)
        except Exception as e:
            print(f"Unable to save data to bucket: {e}", traceback.format_exc())

    def run(self):
        try:
            cleaned_data = self.load_data()
            self.geodf = self.geocode(cleaned_data)
        except Exception as e:
            print(f"Unable to geocode data: {e}", traceback.print_exc())
        finally:
            self.save_data(self.geodf, self.config["geocoded_data_path"])


if __name__ == "__main__":
    path_config = load_config_yaml()["paths"]["WBPPI"]
    path_config["geocoded_data_path"] = "projects/wbppi/geocoded/wbppi_geocoded_data.csv"
    geomapper = WBPPIGeocoder(config=path_config)
    geomapper.run()


## Standardization

In [ ]:
# @title Sector and Subsector standardiser
from tqdm import tqdm


def sector_sub_sector_update(project_data, column):

    """
    This class does a keyword search and maps the keys to likely appropriate
    sector and subsector. May not be the most accurate and efficient method but
    good for a general higher level view.

    Parameters:
        project_data: pandas dataframe with sector and sub sector columns
        column: the column to be considered for identification

    returns:
        pandas dataframe
    """

    project_data[column] = project_data[column].apply(lambda x: str(x).lower())
    new_sector = []
    new_sub_sector = []

    for i, sector in tqdm(enumerate(project_data[column])):
        new_sector.append([])
        new_sub_sector.append([])

        ######  Agriculture   ############################
        if "fish" in sector:
            new_sector[i].append("agriculture")
            new_sub_sector[i].append("fisheries")

        if (
            "crop" in sector
            or "grain" in sector
            or "fruits" in sector
            or "vegetable" in sector
            or "sugarcane" in sector
            or "cotton" in sector
        ):
            new_sector[i].append("agriculture")
            new_sub_sector[i].append("crops")

        if (
            "agribusiness" in sector
            or "breweries" in sector
            or "agroindustry" in sector
            or "storage" in sector
        ):
            if "apartment" in sector:
                pass
            else:
                new_sector[i].append("agriculture")
                new_sub_sector[i].append("agribusiness")

        if (
            "coffee" in sector
            or "cocoa" in sector
            or "tea" in sector
            or "horti" in sector
        ):
            new_sector[i].append("agriculture")
            new_sub_sector[i].append("crops")

        if "forest" in sector:
            new_sector[i].append("agriculture")
            new_sub_sector[i].append("forestry")

        if "livestock" in sector or "animal" in sector or "poultry" in sector:
            new_sector[i].append("agriculture")
            new_sub_sector[i].append("livestock")

        if (
            "irrigation" in sector
            or "drainage" in sector
            or ("agri" in sector and "flood" in sector)
            or "natural resources management" in sector
        ):
            new_sector[i].append("agriculture")
            new_sub_sector[i].append("irrigation and drainage")

        if "dairy" in sector or "cattle" in sector:
            new_sector[i].append("agriculture")
            new_sub_sector[i].append("dairy")

        if (
            "agriculture" in sector or "agricultural" in sector or "agro" in sector
        ) and "admin" in sector:
            new_sector[i].append("agriculture")
            new_sub_sector[i].append("administration")

        if "fertilizer" in sector:
            new_sector[i].append("agriculture")
            new_sub_sector[i].append("fertilizer")

        if "wineries" in sector or "sugar and confectionery" in sector:
            if "apartment" in sector:
                pass
            else:
                new_sector[i].append("agriculture")
                new_sub_sector[i].append("agribusiness")

        if "farm products" in sector or "grocery" in sector or "food" in sector:
            if "apartment" in sector:
                pass
            else:
                new_sector[i].append("agriculture")
                new_sub_sector[i].append("agribusiness")

        if (
            "agriculture" in sector
            or "agricultural" in sector
            or "agro" in sector in sector
        ):
            if len(new_sector[i]) == 0:
                new_sector[i].append("agriculture")
                new_sub_sector[i].append("other")

        ####### Energy ##################
        if ("energy" in sector or "electricity" in sector) and (
            "transmission" in sector or "distribution" in sector
        ):
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("transmission and distribution")

        if (
            "distribution and transmission" in sector
            or "power" in sector
            or "heating" in sector
            or "energy utility" in sector
            or ("highway" in sector and "energy" in sector)
            or "electrification" in sector
            or "electric substation" in sector
        ):
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("transmission and distribution")

        if "efficiency" in sector:
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("efficiency and conservation")

        if (
            "electricity generation" in sector
            or "electricity" in sector
            or "watt" in sector
        ):
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("electricity generation")

        if (
            "oil" in sector
            or "gas" in sector
            or "fuel" in sector
            or "refining" in sector
            or "petrochemical" in sector
            or "petroleum" in sector
        ):
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("oil and gas")

        if "hydro" in sector:
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("hydro")

        if "solar" in sector:
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("solar")

        if "wind" in sector:
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("wind")

        if "biomass" in sector or "bioenergy" in sector:
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("biomass")

        if "thermal" in sector or "heating" in sector:
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("thermal")

        if "geothermal" in sector:
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("geothermal")

        if (
            "renewable" in sector
            or "conventional" in sector
            or "waste to energy" in sector
            or "lowcarbon" in sector
            or "clean energy" in sector
        ):
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("renewable")

        if "coal" in sector and "energy" in sector and not "non" in sector:
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("non-renewable")

        if "energy" in sector and "admin" in sector:
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("administration")

        if (
            "gold" in sector
            or "aluminum" in sector
            or "tungsten" in sector
            or "copper" in sector
            or "diamonds" in sector
        ):
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("mining")

        if (
            "quar" in sector
            or "nickel" in sector
            or "gypsum" in sector
            or "silver" in sector
            or "zinc" in sector
            or "coal" in sector
        ):
            new_sector[i].append("energy and extractives")
            new_sub_sector[i].append("mining")

        if "energy" in sector:
            if len(new_sector[i]) == 0:
                new_sector[i].append("energy and extractives")
                new_sub_sector[i].append("other")

        ###### health ###########
        if (
            "health" in sector
            and ("construction" in sector or "facility" in sector)
            or "hospital" in sector
            or "nursing" in sector
            or "clinic" in sector
        ):
            new_sector[i].append("health")
            new_sub_sector[i].append("construction and facility")

        if "health" in sector and "admin" in sector:
            new_sector[i].append("health")
            new_sub_sector[i].append("administration")

        if (
            "health" in sector
            and (
                "system development" in sector
                or "mother" in sector
                or "child" in sector
            )
            or "care" in sector
            or "health programs" in sector
        ):
            new_sector[i].append("health")
            new_sub_sector[i].append("healthcare services")

        if "nutrition" in sector:
            new_sector[i].append("health")
            new_sub_sector[i].append("disease prevention and control")

        if "disease" in sector:
            new_sector[i].append("health")
            new_sub_sector[i].append("disease prevention and control")

        if (
            ("pharmaceuticals" and "medicine" in sector)
            or "medi" in sector
            or "pharmaceutical" in sector
        ):
            new_sector[i].append("health")
            new_sub_sector[i].append("medicine")

        if "health" in sector:
            if len(new_sector[i]) == 0:
                new_sector[i].append("health")
                new_sub_sector[i].append("other")

        ########### transport ##################
        if (
            "waterway" in sector
            or ("water" in sector and "transport" in sector)
            or "harbor" in sector
            or " port" in sector
        ):
            if (
                "export" in sector
                or "import" in sector
                or "support" in sector
                or "airport" in sector
                or "sports" in sector
                or "surface transport" in sector
            ):
                pass
            else:
                new_sector[i].append("transport")
                new_sub_sector[i].append("ports/waterways")

        if (
            "ports" in sector
            or "ship building and repairing" in sector
            or "marine" in sector
        ):
            if (
                "export" in sector
                or "import" in sector
                or "support" in sector
                or "airport" in sector
                or "sports" in sector
                or "surface transport" in sector
            ):
                pass
            else:
                new_sector[i].append("transport")
                new_sub_sector[i].append("ports/waterways")

        if (
            "aviation" in sector
            or ("air" in sector and "transport" in sector)
            or "airport" in sector
        ):
            new_sector[i].append("transport")
            new_sub_sector[i].append("aviation")

        if "railway" in sector or "train" in sector or "rail" in sector:
            if "training" in sector or "light" in sector:
                pass
            else:
                new_sector[i].append("transport")
                new_sub_sector[i].append("railways")

        if (
            "road" in sector
            or "highway" in sector
            or "logging" in sector
            or "tunnel" in sector
            or "bridge" in sector
            or "motorway" in sector
        ):
            new_sector[i].append("transport")
            new_sub_sector[i].append("roads")

        if "transport" in sector and "admin" in sector:
            new_sector[i].append("transport")
            new_sub_sector[i].append("administration")

        if "urban mobility" in sector:
            new_sector[i].append("transport")
            new_sub_sector[i].append("administration")

        if "passenger" in sector or "sidewalk" in sector or "parking lot" in sector:
            new_sector[i].append("transport")
            new_sub_sector[i].append("services")

        if "freight" in sector:
            new_sector[i].append("transport")
            new_sub_sector[i].append("services")

        if "bus and heavy commercial vehicle" in sector:
            new_sector[i].append("transport")
            new_sub_sector[i].append("services")

        if "transport" in sector:
            if len(new_sector[i]) == 0:
                new_sector[i].append("transport")
                new_sub_sector[i].append("other")

        ############# Education ###########
        if (
            "workforce development" in sector
            or "skill" in sector
            or "vocation" in sector
            or "training" in sector
        ):
            new_sector[i].append("education")
            new_sub_sector[i].append("workforce devlopment/skills")

        if "school" in sector:
            if "primary school" in sector:
                pass
            else:
                new_sector[i].append("education")
                new_sub_sector[i].append("secondary education")

        if "college" in sector or "universi" in sector:
            new_sector[i].append("education")
            new_sub_sector[i].append("tertiary education")

        if (
            "childhood development" in sector
            or "university" in sector
            or "primary school" in sector
        ):
            new_sector[i].append("education")
            new_sub_sector[i].append("childhood education")

        if "education" in sector:
            if "tertiary" in sector or "higher" in sector:
                new_sector[i].append("education")
                new_sub_sector[i].append("tertiary education")
            elif "secondary" in sector:
                new_sector[i].append("education")
                new_sub_sector[i].append("secondary education")
            elif "primary" in sector:
                new_sector[i].append("education")
                new_sub_sector[i].append("primary education")
            elif "childhood" in sector:
                new_sector[i].append("education")
                new_sub_sector[i].append("childhood education")
            elif (
                "adult" in sector
                or "basic" in sector
                or "continuing" in sector
                or "nonformal education" in sector
                or "compensatory" in sector
            ):
                new_sector[i].append("education")
                new_sub_sector[i].append("adult, basic or continuing education")
            elif "admin" in sector or "institution" in sector:
                new_sector[i].append("education")
                new_sub_sector[i].append("administration")
            elif "research" in sector:
                new_sub_sector[i].append("research")
                new_sector[i].append("education")
            else:
                if len(new_sector[i]) == 0:
                    new_sector[i].append("education")
                    new_sub_sector[i].append("other")

        ####### Financial ###########
        if "bank" in sector and not "non" in sector:
            new_sector[i].append("financial")
            new_sub_sector[i].append("banking")

        if "preinvestment" in sector:
            new_sector[i].append("financial")
            new_sub_sector[i].append("banking")

        if ("non" in sector and "bank" in sector) or "noninvestment" in sector:
            new_sector[i].append("financial")
            new_sub_sector[i].append("non-bank finance")

        if "insurance" in sector or "pension" in sector:
            new_sector[i].append("financial")
            new_sub_sector[i].append("insurance and pension")

        if "capital market" in sector or (
            (
                "equity" in sector
                or "debt" in sector
                or "investment" in sector
                or "portfolio" in sector
                or "capital" in sector
            )
            and "fund" in sector
        ):
            new_sector[i].append("financial")
            new_sub_sector[i].append("capital market")

        if (
            "credit lines" in sector
            or "loans" in sector
            or "lending" in sector
            or "small business fund" in sector
            or "mortgage" in sector
            or "fund" in sector
        ):
            new_sector[i].append("financial")
            new_sub_sector[i].append("credit lines/loan")

        if (
            ("financial" in sector or "finance" in sector)
            and "admin" in sector
            or "privatization" in sector
            or "remittance" in sector
            or "regulation and supervision" in sector
        ):
            new_sector[i].append("financial")
            new_sub_sector[i].append("administration")

        if (
            "agricultural markets" in sector or "agricultural credit" in sector
        ) or "extension" in sector:
            new_sector[i].append("financial")
            new_sub_sector[i].append("agricultural markets")

        if "economic management" in sector or "consumer" in sector:
            new_sector[i].append("financial")
            new_sub_sector[i].append("administration")

        if "payment" in sector or "inclus" in sector:
            new_sector[i].append("financial")
            new_sub_sector[i].append("services")

        if (
            "distressed assets" in sector
            or "decentralization" in sector
            or "credit information services" in sector
            or "credit reporting and secured transactions" in sector
        ):
            new_sector[i].append("financial")
            new_sub_sector[i].append("administration")

        if (
            "mutual fund" in sector
            or "exchanges (trading systems)" in sector
            or "fund management" in sector
            or "derivatives (includes risk management entities)" in sector
            or "leasing" in sector
        ):
            new_sector[i].append("financial")
            new_sub_sector[i].append("capital market")

        if "financial" in sector or "finance" in sector:
            if len(new_sector[i]) == 0:
                new_sector[i].append("financial")
                new_sub_sector[i].append("other")

        ####### water, sanitation and waste management ############
        if (
            "sanitation" in sector
            or "sewerage" in sector
            or ("sanit" in sector and "water" in sector)
            or "sewer" in sector
        ):
            new_sector[i].append("water, sanitation and waste management")
            new_sub_sector[i].append("sanitation and sewerage")

        if ("water" in sector and "supply" in sector) or "water" in sector:
            new_sector[i].append("water, sanitation and waste management")
            new_sub_sector[i].append("supply")

        if "water" in sector and "admin" in sector:
            new_sector[i].append("water, sanitation and waste management")
            new_sub_sector[i].append("administration")

        if (
            "water" in sector and "waste" in sector
        ) or "sewage treatment plant" in sector:
            new_sector[i].append("water, sanitation and waste management")
            new_sub_sector[i].append("sanitation and sewerage")

        if "waste" in sector:
            if "solid" in sector or "municipal" in sector:
                new_sector[i].append("water, sanitation and waste management")
                new_sub_sector[i].append("muncipal waste")
            if "water" in sector or "liquid" in sector:
                new_sector[i].append("water, sanitation and waste management")
                new_sub_sector[i].append("liquid waste")
            else:
                if len(new_sector[i]) == 0:
                    new_sector[i].append("water, sanitation and waste management")
                    new_sub_sector[i].append("other")

        ###### industry , trade and services ########
        if (
            "housing" in sector
            or "house" in sector
            or "residential building" in sector
            or "resident" in sector
            or "apartment" in sector
            or "dormitory" in sector
        ):
            if "warehouse" in sector:
                pass
            else:
                new_sector[i].append("industry, trade and services")
                new_sub_sector[i].append("housing")

        if (
            ("urban" in sector and "development" in sector)
            or "modernization" in sector
            or "urban developm" in sector
            or "urban management" in sector
            or "rural" in sector
            or "light" in sector
            or "lighting" in sector
        ):
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("urban and rural development")

        if (
            "building" in sector
            and "residential" in sector
            and not "non" in sector
            or "home" in sector
        ):
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("housing")

        if "building" in sector:
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("urban and rural development")

        if (
            "fabric mills" in sector
            or "finishing (dyeing, printing, finishing, etc.)" in sector
        ):
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("manufacturing")

        if "broker/dealer" in sector or "distribution business" in sector:
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("trade")

        if "manufacturing" in sector:
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("manufacturing")

        if (
            "industry" in sector
            or "trade" in sector
            or "services" in sector
            or "industries" in sector
        ) and "administration" in sector:
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("administration")

        if (
            "tourism" in sector
            or "tour" in sector
            or "amusement park" in sector
            or "travel" in sector
        ):
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("tourism")

        if "hotel" in sector or "motel" in sector:
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("hotel")

        if "mining" in sector:
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("mining")

        if (
            "sme" in sector
            or "small scale enterprise" in sector
            or "industrial" in sector
            or "enterprise" in sector
            or "industries" in sector
            or "sector development" in sector
        ):
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("industry and development")

        if (
            "trade" in sector
            or "sugar and confectionery" in sector
            or "b2c" in sector
            or "sales" in sector
            or "mall" in sector
        ):
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("trade")

        if "industry" in sector or "commercial" in sector:
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("industry and development")

        if (
            "rental" in sector
            or "packaging" in sector
            or "printing" in sector
            or "postal" in sector
            or "service" in sector
        ):
            if "govern" in sector:
                pass
            else:
                new_sector[i].append("industry, trade and services")
                new_sub_sector[i].append("services")

        if (
            "aerospace product and parts" in sector
            or "cement" in sector
            or "concrete pipes" in sector
            or "garment and apparel" in sector
            or "furniture" in sector
            or "ceramic tiles" in sector
            or "glass products" in sector
            or "bricks" in sector
            or "coated products" in sector
        ):
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("manufacturing")

        if (
            "alkalies and chlorine" in sector
            or "chemical" in sector
            or "carbon black" in sector
        ):
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("chemical")

        if "agency reform" in sector:
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("administration")

        if "automotive" in sector:
            new_sector[i].append("industry, trade and services")
            new_sub_sector[i].append("manufacturing")

        if (
            "wood products" in sector
            or "sawmilling" in sector
            or "spinning" in sector
            or "switch" in sector
            or "soft drink" in sector
            or "paper" in sector
            or "paint" in sector
            or "plastic" in sector
            or "soap" in sector
            or "shoes" in sector
            or "steel" in sector
            or "pipelines" in sector
            or "mill" in sector
            or "ceramic" in sector
            or "pottery" in sector
            or "metal" in sector
            or "nylon" in sector
            or "iron" in sector
            or "motor" in sector
        ):

            if "environ" in sector:
                pass
            else:
                new_sector[i].append("industry, trade and services")
                new_sub_sector[i].append("manufacturing")

        if (
            "utilities" in sector
            or "arena" in sector
            or "stadium" in sector
            or "office" in sector
            or "entertainment" in sector
        ):
            if "environ" in sector or "water" in sector or "social safety" in sector:
                pass
            else:
                new_sector[i].append("industry, trade and services")
                new_sub_sector[i].append("infrastructure")

        if "fitness" in sector or "sports" in sector or "recreation center":
            if "environ" in sector or "water" in sector or "social safety" in sector:
                pass
            else:
                new_sector[i].append("industry, trade and services")
                new_sub_sector[i].append("infrastructure")

        if "infrastructure" in sector:
            if "environ" in sector or "water" in sector or "social safety" in sector:
                pass
            else:
                new_sector[i].append("industry, trade and services")
                new_sub_sector[i].append("infrastructure")

        if "composite infrastructure" in sector or "construction" in sector:
            if "environ" in sector or "water" in sector:
                pass
            else:
                new_sector[i].append("industry, trade and services")
                new_sub_sector[i].append("infrastructure")

        ###### ICT ############
        if (
            ("ict" in sector or "information and communications technolog" in sector)
            and "services" in sector
            or "broadband" in sector
        ):
            new_sector[i].append("ict")
            new_sub_sector[i].append("services")

        if (
            ("ict" in sector or "information and communications technolog" in sector)
            and "infrastructure" in sector
            or "it enabled services" in sector
        ):
            new_sector[i].append("ict")
            new_sub_sector[i].append("infrastructure")

        if (
            "ict" in sector or "information and communications technolog" in sector
        ) and "administration" in sector:
            new_sector[i].append("ict")
            new_sub_sector[i].append("administration")

        if "computer systems design and related services" in sector:
            new_sector[i].append("ict")
            new_sub_sector[i].append("infrastructure")

        if "electrical machinery" in sector:
            new_sector[i].append("ict")
            new_sub_sector[i].append("infrastructure")

        if (
            ("tele" in sector)
            or "mobile" in sector
            or "internet" in sector
            or "communication" in sector
        ):
            new_sector[i].append("ict")
            new_sub_sector[i].append("telecommunication")

        if "information vendor" in sector:
            new_sector[i].append("ict")
            new_sub_sector[i].append("administration")

        if "data center" in sector or "data centre" in sector:
            new_sector[i].append("ict")
            new_sub_sector[i].append("data center")

        if "ict" in sector or "information and communication" in sector:
            if len(new_sector[i]) == 0:
                new_sector[i].append("ict")
                new_sub_sector[i].append("other")

        ###### social #############
        if "social" in sector and "admin" in sector:
            new_sector[i].append("social protection")
            new_sub_sector[i].append("administration")

        if (
            "social" in sector
            and ("investment" in sector or "service" in sector)
            or "nondepository" in sector
        ):
            new_sector[i].append("social protection")
            new_sub_sector[i].append("investment/service")

        if "labor" in sector or "employ" in sector:
            new_sector[i].append("social protection")
            new_sub_sector[i].append("investment/service")

        if (
            "social protection" in sector
            or "compensatory" in sector
            or "social safety" in sector
        ):
            new_sector[i].append("social protection")
            new_sub_sector[i].append("programmes, schemes and safety measures")

        if "social" in sector:
            if len(new_sector[i]) == 0:
                new_sector[i].append("social protection")
                new_sub_sector[i].append("other")

        ########## natural disaster/environment and tech ################
        if "flood protection" in sector or "flood control" in sector:
            new_sector[i].append("environment")
            new_sub_sector[i].append("disastor management")

        if "disaster" in sector:
            new_sector[i].append("environment")
            new_sub_sector[i].append("disastor management")

        if (
            "environment" in sector
            or "natural resources" in sector
            or "pollution" in sector
        ):
            new_sector[i].append("environment")
            new_sub_sector[i].append("environmental protection")

        if "research" in sector or "science and technology" in sector:
            new_sector[i].append("science and technology and research")
            new_sub_sector[i].append("science and technology and research")

        if "multi" in sector and "sector" in sector:
            new_sector[i].append("multi sector")
            new_sub_sector[i].append("multi sector")

        ####### Public Administration ############
        if (
            "central" in sector
            and "government" in sector
            or "regional integration" in sector
        ):
            new_sector[i].append("public administration")
            new_sub_sector[i].append("central government")

        if "sub-national" in sector and "government" in sector:
            new_sector[i].append("public administration")
            new_sub_sector[i].append("sub-national government")

        if "civil service reform" in sector or "economic management" in sector:
            new_sector[i].append("public administration")
            new_sub_sector[i].append("central government")

        if "subnational" in sector:
            new_sector[i].append("public administration")
            new_sub_sector[i].append("sub-national government")

        if (
            "aids" in sector
            or "com signatory" in sector
            or "government services" in sector
        ):
            new_sector[i].append("public administration")
            new_sub_sector[i].append("public sector management")

        if (
            "public" in sector
            and "management" in sector
            or "institutional development" in sector
            or "capacity development" in sector
        ):
            new_sector[i].append("public administration")
            new_sub_sector[i].append("public sector management")

        if "law" in sector or "policy" in sector or "adjustment" in sector:
            new_sector[i].append("public administration")
            new_sub_sector[i].append("laws and policies")

        if (
            "justice" in sector
            or "judiciary" in sector
            or "judicial" in sector
            or "justice" in sector
        ):
            new_sector[i].append("public administration")
            new_sub_sector[i].append("judiciary")

        if "public administration" in sector:
            if len(new_sector[i]) == 0:
                new_sector[i].append("public administration")
                new_sub_sector[i].append("other")

    ####### Modifications ############
    project_data["identified_sector"] = new_sector
    project_data["identified_subsector"] = new_sub_sector

    sec_sub_sec_tupple = []
    for i in range(project_data.shape[0]):
        temp = []
        for j in range(len(project_data.iloc[i, -2])):
            temp.append((project_data.iloc[i, -2][j], project_data.iloc[i, -1][j]))
        sec_sub_sec_tupple.append(temp)

    project_data["identified_sector_subsector_tuple"] = sec_sub_sec_tupple
    project_data.drop(columns=[column], inplace=True)

    return project_data


time: 4.47 s (started: 2021-07-20 14:19:01 +00:00)


In [ ]:
#@title ###Standardization Script { form-width: "20%" }
import warnings
import traceback
import pandas as pd
import numpy as np

warnings.simplefilter(action="ignore")

# from ppp_scrapers.data_processing.standardization.standardizer import (
#     sector_sub_sector_update,
# )
from ppp_scrapers.utils import load_config_yaml
from ppp_scrapers.utils.bucket import (
    connect_to_buffer_bucket,
    push_csv_to_buffer_bucket,
    read_csv_from_buffer_bucket,
)


class WBPPIStandardiser:
    def __init__(self, **kwargs):
        self.config: dict = kwargs.get("config")
        self.bucket = connect_to_buffer_bucket()
        self.standardised_df: pd.core.frame.DataFrame = None

    def rename_columns(self, df: pd.core.frame.DataFrame):
        renaming_dict = {
            "project_id": "original_id",
            "project_name": "name",
            "status": "status",
            "status_update_date": "status_update_date",
            "ppi_type": "ppi_type",
            "ppi_subtype": "ppi_subtype",
            "country": "country_name",
            "location": "location",
            "sector": "sector",
            "subsector": "subsector",
            "segment": "segment",
            "technology": "technology",
            "investment_amt_usd": "budget",
            "investment_history": "investment_history",
            "capacity_value": "capacity_value",
            "capacity_type_name": "capacity_type_name",
            "company": "company",
            "sponsor": "sponsor",
            "description": "description",
            "related_name": "related_name",
            "related_projects": "related_projects",
            "financial_closure_month": "financial_closure_month",
            "financial_closure_year": "financial_closure_year",
            "estimated_financial_closure_year": "estimated_financial_closure_year",
            "contract_period": "contract_period",
            "termination_year": "termination_year",
            "award_method": "award_method",
            "govt_lvl_granting": "govt_lvl_granting",
            "bid_criteria": "bid_criteria",
            "number_of_bids": "number_of_bids",
            "main_revenue_source": "main_revenue_source",
            "other_revenue_source": "other_revenue_source",
            "unsolicited_proposal": "unsolicited_proposal",
            "public_disclosure": "public_disclosure",
            "description_of_source": "description_of_source",
            "total_equity": "total_equity",
            "total_ownership": "total_ownership",
            "multi_lateral_support": "multi_lateral_support",
            "bilateral": "bilateral",
            "type_of_direct_govt_support": "type_of_direct_govt_support",
            "type_of_direct_govt_support_value": "type_of_direct_govt_support_value",
            "type_of_indirect_govt_support": "type_of_indirect_govt_support",
            "type_of_indirect_govt_support_value": "type_of_indirect_govt_support_value",
            "third_party_support": "third_party_support",
            "source": "source",
            "api_url": "api_url",
            "url": "url",
            "country_code": "country_code",
            "region_name": "region_name",
            "region_code": "region_code",
            "project_or_tender": "project_or_tender",
            "aug_id": "aug_id",
            "label": "location_label",
            "map_coordinates": "map_coordinates",
            "identified_sector": "identified_sector",
            "identified_subsector": "identified_subsector",
            "identified_sector_subsector_tuple": "identified_sector_subsector_tuple",
        }
        return df.rename(columns=renaming_dict)

    def standardise_data(self, df):
        try:
            for col in self.config["columns"]:
                df[col] = df[col].replace([np.nan], "")
            df["industry_info"] = df[self.config["columns"]].apply(
                lambda x: " ".join(x.tolist()), axis=1
            )
            df = sector_sub_sector_update(df, "industry_info")
            df["identified_sector"] = df["identified_sector"].apply(
                lambda x: x[0] if len(x) >= 1 else "other"
            )
            df["identified_subsector"] = df["identified_subsector"].apply(
                lambda x: x[0] if len(x) >= 1 else "other"
            )
            df.drop(columns=["identified_sector_subsector_tuple"], inplace=True)
            df = self.rename_columns(df)
            return df
        except Exception as e:
            print(e)

    def load_data(self):
        try:
            df = read_csv_from_buffer_bucket(
                self.bucket, self.config["geocoded_data_path"]
            )
            return df
        except FileNotFoundError as e:
            print(f"Unable to read file from bucket: {e}", traceback.format_exc())

    def save_data(self, df, save_path):
        try:
            push_csv_to_buffer_bucket(self.bucket, df, save_path)
        except Exception as e:
            print(f"Unable to save data to bucket: {e}", traceback.format_exc())

    def run(self):
        try:
            geocoded_data = self.load_data()
            self.standardised_df = self.standardise_data(geocoded_data)
        except Exception as e:
            print(f"Unable to geocode data: {e}", traceback.print_exc())
        finally:
            self.save_data(self.standardised_df, self.config["standardised_data_path"])


if __name__ == "__main__":
    path_config = load_config_yaml()["paths"]["WBPPI"]
    path_config["geocoded_data_path"] = "projects/wbppi/geocoded/wbppi_geocoded_data.csv"
    path_config["standardised_data_path"] = "projects/wbppi/standardised/wbppi_standardised_data.csv"
    config = {
        "columns": ["sector", "subsector", "segment", "technology"],
        "geocoded_data_path": path_config["geocoded_data_path"],
        "standardised_data_path": path_config["standardised_data_path"],
    }
    standardizer = WBPPIStandardiser(config=config)
    standardizer.run()


Successfully connected to the bucket "psd_data"


8350it [00:00, 11801.63it/s]


Successfully pushed the csv file to "projects/wbppi/standardised/wbppi_standardised_data.csv"
time: 4.32 s (started: 2021-07-20 14:28:10 +00:00)


# Named Entity Recognition Section

In [ ]:
# import spacy
# from spacy import displacy

# # Load English tokenizer, tagger, parser and NER
# nlp = spacy.load("en_core_web_sm")

# # Process whole documents
# for text in df.description.values.tolist()[:2]:
#     # text = ("Kattimuruchchana, Thirappane,")
#     doc = nlp(text)

#     # Analyze syntax
#     print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
#     print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

#     # Find named entities, phrases and concepts
#     for entity in doc.ents:
#         print(entity.text, entity.label_)

#     displacy.render(doc, style='ent', jupyter=True)

time: 2.26 ms (started: 2021-07-22 12:44:07 +00:00)


In [ ]:
doc.ents

(Punta Hermosa, Punta Negra, San Bartolo, Santa María)

time: 8.44 ms (started: 2021-07-20 11:46:35 +00:00)


# New Section

In [8]:
#@title ### Sam.gov Geocoder { form-width: "30%" }
import sys, traceback
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore')

from ppp_scrapers.utils import load_config_yaml
from ppp_scrapers.utils.bucket import (
    read_csv_from_buffer_bucket,
    read_json_from_buffer_bucket,
)
from ppp_scrapers.data_processing.clean.BaseCleaner import BaseCleaner


class cleanSAMGOVCO(BaseCleaner):
    def __init__(self, **kwargs):
        super().__init__()
        self.config: dict = kwargs.get("config")
        self.naics_converter: dict = None

        self.load_extra_dicts()

    def load_extra_dicts(self):
        self.naics_converter = read_json_from_buffer_bucket(
            self.bucket, self.config["path_config"]["naics_code_path"]
        )

    def load_data(self, csv_path: str):
        try:
            raw_csv = read_csv_from_buffer_bucket(self.bucket, csv_path)
            return raw_csv
        except FileNotFoundError as e:
            print(e)

    def filter_naics(self,x):
        try:
            x = str(int(x))
            return self.naics_converter[x]
        except Exception as e:
            return "Not Identified"

    def find_amounts(self,x):
        x = re.findall(
            '(?:(?:\$[0-9,.]+)+(?:\smillion))|(?:\$[0-9.,]+)|(?:[0-9,.]+)', str(x)
        )
        if len(x)>=1:
            try:
                x = [float(re.sub('[$,]','',i)) for i in x]
            except:
                pass
            try:
                x = [float(re.sub('[$,.]','',i)) for i in x if len(i)>1]
            except:
                pass
            try:
                for i in range(len(x)):
                    if 'million' in x[i]:
                        x[i] = float(re.sub('[$milon ]',''))*1000000
            except:
                pass
            if len(x)>=1:
                x = max(x)
            else:
                x = np.nan
        else:
            x = np.nan
        return x

    def rename_cols(self,df):
        return df.rename(
            columns={
                'AAC Code': 'aac_code',
                'Active': 'active',
                'AdditionalInfoLink': 'additional_info_link',
                'ArchiveDate': 'archive_date',
                'ArchiveType': 'archive_type',
                'Award$': 'award_usd',
                'AwardDate': 'award_date',
                'AwardNumber': 'award_number',
                'Awardee': 'awardee',
                'BaseType': 'basetype',
                'CGAC': 'cgac',
                'City': 'city',
                'ClassificationCode': 'classification_code',
                'CountryCode': 'countrycode',
                'Department/Ind.Agency': 'department_or_industry_agency',
                'Description': 'description',
                'FPDS Code': 'fpds_code',
                'Link': 'url',
                'NaicsCode': 'naics_code',
                'NoticeId': 'tender_id',
                'Office': 'office',
                'OrganizationType': 'organization_type',
                'PopCity': 'pop_city',
                'PopCountry': 'pop_country',
                'PopState': 'pop_state',
                'PopStreetAddress': 'pop_street_address',
                'PopZip': 'pop_zip',
                'PostedDate': 'posted_date',
                'PrimaryContactEmail': 'primary_contact_email',
                'PrimaryContactFax': 'primary_contact_fax',
                'PrimaryContactFullname': 'primary_contact_fullname',
                'PrimaryContactPhone': 'primary_contact_phone',
                'PrimaryContactTitle': 'primary_contact_title',
                'ResponseDeadLine': 'response_deadline',
                'SecondaryContactEmail': 'secondary_contact_email',
                'SecondaryContactFax': 'secondary_contact_fax',
                'SecondaryContactFullname': 'secondary_contact_fullname',
                'SecondaryContactPhone': 'secondary_contact_phone',
                'SecondaryContactTitle': 'secondary_contact_title',
                'SetASide': 'set_a_side',
                'SetASideCode': 'set_a_side_code',
                'Sol#': 'solicitation_id',
                'State': 'state',
                'Sub-Tier': 'sub_tier',
                'Title': 'tender_name',
                'Type': 'type',
                'ZipCode': 'zipcode'
            }
        )

    def clean(self, df):
        try:
            df = df[~df["Title"].isnull()]
            df['Title'] = df['Title'].apply(lambda x: x.title() if type(x)==str else x)
            df['Department/Ind.Agency'] = df['Department/Ind.Agency'].apply(lambda x: x.title() if type(x)==str else x)
            df['Sub-Tier'] = df['Sub-Tier'].apply(lambda x: x.title() if type(x)==str else x)

            df['PostedDate'] = pd.to_datetime(df['PostedDate'], errors="coerce", format='%Y-%m-%d', utc=True)
            df['PostedDate'] = df['PostedDate'].astype('datetime64[ns]').dt.date

            df['ArchiveDate'] = pd.to_datetime(df['ArchiveDate'], errors="coerce", format='%Y-%m-%d', utc=True)
            df['ArchiveDate'] = df['ArchiveDate'].astype('datetime64[ns]').dt.date 

            df['ResponseDeadLine'] = pd.to_datetime(df['ResponseDeadLine'], errors="coerce", format='%Y-%m-%d', utc=True)
            df['ResponseDeadLine'] = df['ResponseDeadLine'].astype('datetime64[ns]').dt.date 

            df['City'] = df['City'].apply(lambda x: x.title() if type(x)==str else x)
            df['country_code'] = (
                df['CountryCode']
                .replace([np.nan],"")
                .replace({'US':'USA', 'XKS':'XKX', "Morocco":"MAR"})
            )
            df['country_name'] = df['country_code'].apply(
                lambda x: self.country_code2name_dict[x]
                if x in self.country_code2name_dict.keys()
                else x
            )
            df['region_name'] = df['country_code'].apply(
                lambda x: self.country_code2region_name_dict[x] 
                if x in self.country_code2region_name_dict.keys() 
                else x
            )
            df['region_code'] = df['region_name'].apply(
                lambda x: self.region_name2code_dict[x]
                if x in self.region_name2code_dict.keys()
                else x
            )

            df['investment_amt_usd'] = df['Award$'].apply(
                lambda x: self.find_amounts(x)
            )
            df['investment_amt_usd'] = df['investment_amt_usd'].apply(
                lambda x: x if type(x)==float else np.nan
            )

            df['naics_name'] = df['NaicsCode'].apply(lambda x: self.filter_naics(x))

            df["project_or_tender"] = "T"
            df["source"] = "SAMGOVCO"
            df["aug_id"] = "SAMGOVCO_"+df["NoticeId"].astype(str)
            return self.rename_cols(df)
        except Exception as e:
            print(f"Error cleaning raw JSON file: {e}\n{traceback.print_exc()}")

    def run(self):
        try:
            for key in tqdm(list(self.config["path_config"]["raw_data_paths"].keys())):
                raw_data = self.load_data(self.config["path_config"]["raw_data_paths"][key])
                df = self.clean(raw_data)
                self.save_data(df, self.config["path_config"]["cleaned_data_path"][key])
        except Exception as e:
            print(e,traceback.format_exc())


if __name__ == "__main__":
    path_config = load_config_yaml()["paths"]["SAMGOVCO"]
    config = {"path_config": path_config}
    cleaner = cleanSAMGOVCO(config=config)
    cleaner.run()


Successfully connected to the bucket "psd_data"


  7%|▋         | 2/28 [00:00<00:05,  4.74it/s]

Successfully pushed the csv file to "projects/samgovco/clean/year_wise/FY1970_archived_opportunities.csv"
Successfully pushed the csv file to "projects/samgovco/clean/year_wise/FY1980_archived_opportunities.csv"


 11%|█         | 3/28 [00:00<00:05,  4.24it/s]

Successfully pushed the csv file to "projects/samgovco/clean/year_wise/FY1998_archived_opportunities.csv"


...

 96%|█████████▋| 27/28 [16:21<00:41, 41.37s/it]

Successfully pushed the csv file to "projects/samgovco/clean/year_wise/FY2030_archived_opportunities.csv"


100%|██████████| 28/28 [16:53<00:00, 36.21s/it]

Successfully pushed the csv file to "projects/samgovco/clean/year_wise/ContractOpportunitiesFullCSV.csv"


time: 16min 56s (started: 2021-07-25 12:06:30 +00:00)


In [15]:
path_config = load_config_yaml()["paths"]["SAMGOVCO"]
path_config['cleaned_data_path']

{'1970': 'projects/samgovco/clean/year_wise/FY1970_archived_opportunities.csv',
 '1980': 'projects/samgovco/clean/year_wise/FY1980_archived_opportunities.csv',
 '1998': 'projects/samgovco/clean/year_wise/FY1998_archived_opportunities.csv',
 '1999': 'projects/samgovco/clean/year_wise/FY1999_archived_opportunities.csv',
 '2000': 'projects/samgovco/clean/year_wise/FY2000_archived_opportunities.csv',
 '2001': 'projects/samgovco/clean/year_wise/FY2001_archived_opportunities.csv',
 '2002': 'projects/samgovco/clean/year_wise/FY2002_archived_opportunities.csv',
 '2003': 'projects/samgovco/clean/year_wise/FY2003_archived_opportunities.csv',
 '2004': 'projects/samgovco/clean/year_wise/FY2004_archived_opportunities.csv',
 '2005': 'projects/samgovco/clean/year_wise/FY2005_archived_opportunities.csv',
 '2006': 'projects/samgovco/clean/year_wise/FY2006_archived_opportunities.csv',
 '2007': 'projects/samgovco/clean/year_wise/FY2007_archived_opportunities.csv',
 '2008': 'projects/samgovco/clean/year_w

time: 73.8 ms (started: 2021-07-25 12:26:43 +00:00)


In [47]:
import pandas as pd
import numpy as np
import warnings
import traceback
from tqdm import tqdm

import geopandas

warnings.simplefilter(action="ignore")

from ppp_scrapers.utils.geocode2 import Geocoder
from ppp_scrapers.utils import load_config_yaml
from ppp_scrapers.utils.bucket import (
    connect_to_buffer_bucket,
    push_csv_to_buffer_bucket,
    read_csv_from_buffer_bucket,
)


class SAMGOVCOGeocoder:
    def __init__(self, **kwargs):
        self.config = kwargs.get("config")
        self.bucket = connect_to_buffer_bucket()
        self.geocoder = Geocoder()
        self.geodf: pd.core.frame.DataFrame = None

    def load_data(self,path):
        try:
            df = read_csv_from_buffer_bucket(self.bucket, path)
            return df
        except FileNotFoundError as e:
            print(f"Unable to read file from bucket: {e}", traceback.format_exc())

    def my_geocoder(self, row):
        try:
            point = geopandas.tools.geocode(row, provider="nominatim").geometry.iloc[0]
            return pd.Series(
                {"latitude": point.y, "longitude": point.x, "geometry": point}
            )
        except:
            return None

    def my_geocoderx(self, row):
        try:
            resp = geopandas.tools.geocode(row, provider="nominatim")
            point = resp.geometry.iloc[0]
            return {
                row: pd.Series(
                    {
                        "latitude": point.y,
                        "longitude": point.x,
                        "geometry": point,
                        "address": resp.address[0],
                    }
                )
            }
        except Exception as e:
            return {
                row: pd.Series(
                    {
                        "latitude": np.nan,
                        "longitude": np.nan,
                        "geometry": None,
                        "address": None,
                    }
                )
            }

    def geocode(self, df):
        try:
            df['city'] = df['city'].replace([np.nan],"")
            df['country_code'] = df['country_code'].replace([np.nan],"")
            df['usable_location_string'] = df['city']+', '+df['country_code']

            temp_dict = {}
            for loc in tqdm(df.usable_location_string.dropna().unique().tolist()):
                temp_dict.update(self.my_geocoderx(loc))
                
            df[["latitude", "longitude", "geometry", "address"]] = df.apply(
                lambda x: temp_dict[x["usable_location_string"]]
                if x["usable_location_string"] in temp_dict.keys()
                else None,
                axis=1,
            )

            df.drop(columns=['usable_location_string'],inplace=True)
            return df
        except Exception as e:
            print(traceback.format_exc())

    def save_data(self, df, save_path):
        try:
            push_csv_to_buffer_bucket(self.bucket, df, save_path)
        except Exception as e:
            print(f"Unable to save data to bucket: {e}", traceback.format_exc())

    def run(self):
        try:
            for key in tqdm(
                list(self.config["path_config"]["cleaned_data_path"].keys())[:7]
            ):
                cleaned_data = self.load_data(
                    self.config["path_config"]["cleaned_data_path"][key]
                )
                df = self.geocode(cleaned_data)
                self.save_data(
                    df,
                    self.config["path_config"]["cleaned_data_path"][key].replace(
                        "clean", "geocoded"
                    ),
                )
        except Exception as e:
            print(f"Unable to geocode data: {e}", traceback.print_exc())


if __name__ == "__main__":
    path_config = load_config_yaml()["paths"]["SAMGOVCO"]
    config = {"path_config": path_config}
    geomapper = SAMGOVCOGeocoder(config=config)
    geomapper.run()


  0%|          | 0/2 [00:00<?, ?it/s]

Successfully connected to the bucket "psd_data"



  0%|          | 0/1 [00:00<?, ?it/s]

Successfully pushed the csv file to "projects/samgovco/geocoded/year_wise/FY1970_archived_opportunities.csv"



  0%|          | 0/9 [00:00<?, ?it/s]

...

Successfully pushed the csv file to "projects/samgovco/geocoded/year_wise/FY2000_archived_opportunities.csv"



 86%|████████▌ | 6/7 [01:30<00:21, 21.02s/it]

Successfully pushed the csv file to "projects/samgovco/geocoded/year_wise/FY2001_archived_opportunities.csv"



100%|██████████| 7/7 [06:03<00:00, 51.96s/it]

Successfully pushed the csv file to "projects/samgovco/geocoded/year_wise/FY2002_archived_opportunities.csv"
time: 6min 4s (started: 2021-07-25 16:50:20 +00:00)


In [37]:
from geopandas.tools import geocode

def my_geocoder(row):
    try:
        resp = geocode(row, provider='nominatim')
        point = resp.geometry.iloc[0]
        return pd.Series({'latitude': point.y, 'longitude': point.x, 'geometry': point, 'address':resp.address[0]})
    except:
        return None

def my_geocoderx(row):
    try:
        resp = geocode(row, provider='nominatim')
        point = resp.geometry.iloc[0]
        return {
            row: pd.Series({'latitude': point.y, 'longitude': point.x, 'geometry': point, 'address':resp.address[0]})
        }
    except Exception as e:
        return {
            row: pd.Series({'latitude': np.nan, 'longitude': np.nan, 'geometry': None, 'address':None})
        }

time: 10.2 ms (started: 2021-07-25 16:43:08 +00:00)


In [38]:
result = my_geocoder(df.city.unique().tolist()[0])
result

latitude                                                38.982
longitude                                             -77.1242
geometry                 POINT (-77.12423413116392 38.9819845)
address      Bethesda, Montgomery County, Maryland, United ...
dtype: object

time: 514 ms (started: 2021-07-25 16:43:12 +00:00)


In [39]:
import warnings

warnings.simplefilter(action="ignore")

xdf = df
xdf['city'] = xdf['city'].replace([np.nan],"")
xdf['country_code'] = xdf['country_code'].replace([np.nan],"")
xdf['usable_location_string'] = xdf['city']+', '+xdf['country_code']
# xdf[['latitude', 'longitude', 'geometry', 'address']] = xdf.apply(lambda x: my_geocoder(x['city']), axis=1)
xdf.tail(5)

,tender_id,tender_name,solicitation_id,department_or_industry_agency,cgac,sub_tier,fpds_code,office,aac_code,posted_date,type,basetype,archive_type,archive_date,set_a_side_code,set_a_side,response_deadline,naics_code,classification_code,pop_street_address,pop_city,pop_state,pop_zip,pop_country,active,award_number,award_date,award_usd,awardee,primary_contact_title,primary_contact_fullname,primary_contact_email,primary_contact_phone,primary_contact_fax,secondary_contact_title,secondary_contact_fullname,secondary_contact_email,secondary_contact_phone,secondary_contact_fax,organization_type,state,city,zipcode,countrycode,additional_info_link,url,description,country_code,country_name,region_name,region_code,investment_amt_usd,naics_name,project_or_tender,source,aug_id,usable_location_string
175176,e5d82111eaced6a57db622e5e9a998ca,61--Sole Source Purchase For Schweitzer Relays,DE-RQ65-02WJ59102,NaN,NaN,NaN,NaN,NaN,NaN,2001-12-17,Presolicitation,Presolicitation,manual,NaN,NaN,NaN,NaN,NaN,61,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,"Sharon Gatenby, Contracting Officer, 970-461-7...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,https://beta.sam.gov/opp/e5d82111eaced6a57db62...,The Western Area Power Administration intends ...,,NaN,NaN,NaN,NaN,Not Identified,T,SAMGOVCO,SAMGOVCO_e5d82111eaced6a57db622e5e9a998ca,","
175177,40e98bbcd009386db3486745fa01180c,R--Transportation Safeguards Training Center (...,DE-RP04-02AL67577,NaN,NaN,NaN,NaN,NaN,NaN,2001-11-09,Presolicitation,Presolicitation,manual,NaN,NaN,NaN,2001-12-03,NaN,R,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,"William Mccullough, Contracting Officer, 505-8...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,https://beta.sam.gov/opp/40e98bbcd009386db3486...,DOE/AL will be procuring a range of logistical...,,NaN,NaN,NaN,NaN,Not Identified,T,SAMGOVCO,SAMGOVCO_40e98bbcd009386db3486745fa01180c,","
175178,55dd315c0ac10ce8bcfdfb2b33d4b900,R--Doe-Wide Nepa Contracts,DE-RP04-02AL67464,NaN,NaN,NaN,NaN,NaN,NaN,2001-10-31,Modification/Amendment/Cancel,Modification/Amendment/Cancel,manual,NaN,NaN,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,"William Mccullough, Contracting Officer, 505-8...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,https://beta.sam.gov/opp/55dd315c0ac10ce8bcfdf...,Small Business Sources Sought for the Preparat...,,NaN,NaN,NaN,NaN,Not Identified,T,SAMGOVCO,SAMGOVCO_55dd315c0ac10ce8bcfdfb2b33d4b900,","
175179,ed172bb4789985845b0c44f55c2bf102,R--Doe-Wide Nepa Contracts,DE-RP04-02AL67464,NaN,NaN,NaN,NaN,NaN,NaN,2001-10-31,Modification/Amendment/Cancel,Modification/Amendment/Cancel,manual,NaN,NaN,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,"William Mccullough, Contracting Officer, 505-8...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,https://beta.sam.gov/opp/ed172bb4789985845b0c4...,Small Business Sources Sought for the Preparat...,,NaN,NaN,NaN,NaN,Not Identified,T,SAMGOVCO,SAMGOVCO_ed172bb4789985845b0c44f55c2bf102,","
175180,32411c090b415a7af47339797ef81107,Z--Columbia River Corridor Closure Project,DE-RP06-02RL14300,NaN,NaN,NaN,NaN,NaN,NaN,2001-10-19,Presolicitation,Presolicitation,manual,NaN,NaN,NaN,NaN,NaN,Z,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,"George Champlain, Contracting Officer, 509-376...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,https://beta.sam.gov/opp/32411c090b415a7af4733...,"As required by FAR 5.203, this is formal notic...",,NaN,NaN,NaN,NaN,Not Identified,T,SAMGOVCO,SAMGOVCO_32411c090b415a7af47339797ef81107,","


time: 173 ms (started: 2021-07-25 16:43:20 +00:00)


In [40]:
from tqdm import tqdm
from pprint import pprint

temp_dict = {}
for loc in tqdm(xdf.usable_location_string.dropna().unique().tolist()[:20]):
    temp_dict.update(my_geocoderx(loc))

temp_dict

100%|██████████| 20/20 [00:10<00:00,  1.97it/s]


{'Albuquerque, USA': latitude                                               35.0841
 longitude                                             -106.651
 geometry                        POINT (-106.650985 35.0841034)
 address      Albuquerque, Bernalillo County, New Mexico, Un...
 dtype: object,
 'Apo, USA': latitude                                               42.8937
 longitude                                             -78.8384
 geometry                  POINT (-78.8384486339416 42.8936691)
 address      Appliance Plus Outlet, Buffalo, Erie County, N...
 dtype: object,
 'Atlanta, USA': latitude                                             33.749
 longitude                                          -84.3903
 geometry              POINT (-84.39026440000001 33.7489924)
 address      Atlanta, Fulton County, Georgia, United States
 dtype: object,
 'Barksdale Afb, USA': latitude                                               32.4971
 longitude                                             -93.610

time: 10.2 s (started: 2021-07-25 16:43:40 +00:00)


In [19]:
xdf[['latitude', 'longitude', 'geometry', 'address']] = xdf.apply(
    lambda x: temp_dict[x['usable_location_string']] 
    if x['usable_location_string'] in temp_dict.keys()
    else None, 
    axis=1
)
xdf

,tender_id,tender_name,solicitation_id,department_or_industry_agency,cgac,sub_tier,fpds_code,office,aac_code,posted_date,type,basetype,archive_type,archive_date,set_a_side_code,set_a_side,response_deadline,naics_code,classification_code,pop_street_address,pop_city,pop_state,pop_zip,pop_country,active,award_number,award_date,award_usd,awardee,primary_contact_title,primary_contact_fullname,primary_contact_email,primary_contact_phone,primary_contact_fax,secondary_contact_title,secondary_contact_fullname,secondary_contact_email,secondary_contact_phone,secondary_contact_fax,organization_type,state,city,zipcode,countrycode,additional_info_link,url,description,country_code,country_name,region_name,region_code,investment_amt_usd,naics_name,project_or_tender,source,aug_id,usable_location_string,latitude,longitude,geometry,address
0,f46824eb239941a7ae2e32f0171b0d4e,Healthy Aging In Neighborhoods Of Diversity Ac...,HHS-NIH-NIDA(AG)-SBSS-75N95021Q00251,"Health And Human Services, Department Of",75.0,National Institutes Of Health,7529,NATIONAL INSTITUTES OF HEALTH NIDA,75N950,2021-06-22,Sources Sought,Sources Sought,auto15,2021-07-21,NaN,NaN,2021-07-06,541611,R499,NaN,NaN,NaN,NaN,USA,Yes,NaN,NaN,NaN,NaN,NaN,Fred Ettehadieh,Fred.Ettehadieh@nih.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OFFICE,MD,Bethesda,20892,USA,NaN,https://beta.sam.gov/opp/f46824eb239941a7ae2e3...,This is a Small Business Sources Sought notice...,USA,U.S.,North America,NAC,NaN,Administrative Management and General Manageme...,T,SAMGOVCO,SAMGOVCO_f46824eb239941a7ae2e32f0171b0d4e,"Bethesda, USA",38.981985,-77.124234,POINT (-77.12423413116392 38.9819845),"Bethesda, Montgomery County, Maryland, United ..."
1,8c98677a2f8e4de48bc5759edba78f91,Home Living Essentials & Mattresses,140A2321Q0424,"Interior, Department Of The",14.0,Bureau Of Indian Affairs,1450,INDIAN EDUCATION ACQUISITION OFFICE,140A23,2021-06-22,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,auto15,2021-07-10,ISBEE,Indian Small Business Economic Enterprise (ISB...,2021-06-25,337125,7105,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,"Begay, Janice",Janice.Begay@bie.edu,7857498419,7858302760,NaN,NaN,NaN,NaN,NaN,OFFICE,NM,Albuquerque,87104,USA,NaN,https://beta.sam.gov/opp/8c98677a2f8e4de48bc57...,Quotes must be submitted to janice.begay@bie.e...,USA,U.S.,North America,NAC,NaN,Household Furniture (except Wood and Metal) Ma...,T,SAMGOVCO,SAMGOVCO_8c98677a2f8e4de48bc5759edba78f91,"Albuquerque, USA",35.084103,-106.650985,POINT (-106.650985 35.0841034),"Albuquerque, Bernalillo County, New Mexico, Un..."
2,3340d5629a3d4c6c98eb8f4d3eefd9a4,Fan Coil,N0024421Q0082,Dept Of Defense,17.0,Dept Of The Navy,1700,NAVSUP FLT LOG CTR SAN DIEGO,N00244,2021-06-22,Solicitation,Solicitation,auto15,2021-07-09,NaN,NaN,2021-06-24,333415,4120,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Telephone: 6195565519,charles.e.fletcher1@navy.mil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OFFICE,CA,San Diego,92136,USA,NaN,https://beta.sam.gov/opp/3340d5629a3d4c6c98eb8...,EXTREMELY URGENT BRAND NAME OR EQUAL ALL OR NO...,USA,U.S.,North America,NAC,NaN,Air-Conditioning and Warm Air Heating Equipmen...,T,SAMGOVCO,SAMGOVCO_3340d5629a3d4c6c98eb8f4d3eefd9a4,"San Diego, USA",32.717420,-117.162773,POINT (-117.1627728 32.7174202),"San Diego, San Diego County, California, Unite..."
3,a8cae1fa067f497581061dbe857cfd18,Abeyance Cryo Solutions A440 Ln2 Vapor Freezer,HHS-NIH-NIDA(AG)-SBSS-75N95021Q00250,"Health And Human Services, Department Of",75.0,National Institutes Of Health,7529,NATIONAL INSTITUTES OF HEALTH NIDA,75N950,2021-06-22,Sources Sought,Sources Sought,auto15,2021-07-21,NaN,NaN,2021-07-06,333415,6640,NaN,NaN,NaN,NaN,USA,Yes,NaN,NaN,NaN,NaN,NaN,Fred Ettehadieh,Fred.Ettehadieh@nih.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OFFICE,MD,Bethesda,20892,USA,NaN,https://beta.sam.gov/opp/a8cae1fa067f497581061...,This is a Small Business Sources Sought notice...,USA,U.S.,North America,NAC,NaN,Air-Conditioning and Warm Air Heating Equipmen...,T,SAMGOVCO,SAMGOVCO_a8cae1fa067f497581061dbe857

time: 22 s (started: 2021-07-25 15:56:39 +00:00)


In [21]:
push_csv_to_buffer_bucket(bucket, xdf, 'projects/samgovco/geocoded/year_wise/ContractOpportunitiesFullCSV.csv')

Successfully pushed the csv file to "projects/samgovco/geocoded/year_wise/ContractOpportunitiesFullCSV.csv"
time: 24.1 s (started: 2021-07-25 16:00:05 +00:00)


In [25]:
from ppp_scrapers.data_processing.standardization.standardizer import (
    sector_sub_sector_update,
)
rdf = xdf
rdf['info'] = rdf['tender_name'].astype(str)+" "+rdf['naics_name'].astype(str)
rdf = sector_sub_sector_update(xdf, 'info')
rdf.drop(columns=['info'],inplace=True)
rdf

175181it [00:10, 15984.88it/s]


,tender_id,tender_name,solicitation_id,department_or_industry_agency,cgac,sub_tier,fpds_code,office,aac_code,posted_date,type,basetype,archive_type,archive_date,set_a_side_code,set_a_side,response_deadline,naics_code,classification_code,pop_street_address,pop_city,pop_state,pop_zip,pop_country,active,award_number,award_date,award_usd,awardee,primary_contact_title,primary_contact_fullname,primary_contact_email,primary_contact_phone,primary_contact_fax,secondary_contact_title,secondary_contact_fullname,secondary_contact_email,secondary_contact_phone,secondary_contact_fax,organization_type,state,city,zipcode,countrycode,additional_info_link,url,description,country_code,country_name,region_name,region_code,investment_amt_usd,naics_name,project_or_tender,source,aug_id,usable_location_string,latitude,longitude,geometry,address,new_sector,new_subsector,sector_subsector_tuple
0,f46824eb239941a7ae2e32f0171b0d4e,Healthy Aging In Neighborhoods Of Diversity Ac...,HHS-NIH-NIDA(AG)-SBSS-75N95021Q00251,"Health And Human Services, Department Of",75.0,National Institutes Of Health,7529,NATIONAL INSTITUTES OF HEALTH NIDA,75N950,2021-06-22,Sources Sought,Sources Sought,auto15,2021-07-21,NaN,NaN,2021-07-06,541611,R499,NaN,NaN,NaN,NaN,USA,Yes,NaN,NaN,NaN,NaN,NaN,Fred Ettehadieh,Fred.Ettehadieh@nih.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OFFICE,MD,Bethesda,20892,USA,NaN,https://beta.sam.gov/opp/f46824eb239941a7ae2e3...,This is a Small Business Sources Sought notice...,USA,U.S.,North America,NAC,NaN,administrative management and general manageme...,T,SAMGOVCO,SAMGOVCO_f46824eb239941a7ae2e32f0171b0d4e,"Bethesda, USA",38.981985,-77.124234,POINT (-77.12423413116392 38.9819845),"Bethesda, Montgomery County, Maryland, United ...","[health, health, industry, trade and services,...","[administration, medicine, services, infrastru...","[((industry, trade and services, services), h)..."
1,8c98677a2f8e4de48bc5759edba78f91,Home Living Essentials & Mattresses,140A2321Q0424,"Interior, Department Of The",14.0,Bureau Of Indian Affairs,1450,INDIAN EDUCATION ACQUISITION OFFICE,140A23,2021-06-22,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,auto15,2021-07-10,ISBEE,Indian Small Business Economic Enterprise (ISB...,2021-06-25,337125,7105,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,"Begay, Janice",Janice.Begay@bie.edu,7857498419,7858302760,NaN,NaN,NaN,NaN,NaN,OFFICE,NM,Albuquerque,87104,USA,NaN,https://beta.sam.gov/opp/8c98677a2f8e4de48bc57...,Quotes must be submitted to janice.begay@bie.e...,USA,U.S.,North America,NAC,NaN,household furniture (except wood and metal) ma...,T,SAMGOVCO,SAMGOVCO_8c98677a2f8e4de48bc5759edba78f91,"Albuquerque, USA",35.084103,-106.650985,POINT (-106.650985 35.0841034),"Albuquerque, Bernalillo County, New Mexico, Un...","[industry, trade and services, industry, trade...","[housing, housing, manufacturing, manufacturin...","[((industry, trade and services, housing), h),..."
2,3340d5629a3d4c6c98eb8f4d3eefd9a4,Fan Coil,N0024421Q0082,Dept Of Defense,17.0,Dept Of The Navy,1700,NAVSUP FLT LOG CTR SAN DIEGO,N00244,2021-06-22,Solicitation,Solicitation,auto15,2021-07-09,NaN,NaN,2021-06-24,333415,4120,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Telephone: 6195565519,charles.e.fletcher1@navy.mil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OFFICE,CA,San Diego,92136,USA,NaN,https://beta.sam.gov/opp/3340d5629a3d4c6c98eb8...,EXTREMELY URGENT BRAND NAME OR EQUAL ALL OR NO...,USA,U.S.,North America,NAC,NaN,air-conditioning and warm air heating equipmen...,T,SAMGOVCO,SAMGOVCO_3340d5629a3d4c6c98eb8f4d3eefd9a4,"San Diego, USA",32.717420,-117.162773,POINT (-117.1627728 32.7174202),"San Diego, San Diego County, California, Unite...","[energy and extractives, energy and extractive...","[transmission and distribution, oil and gas, t...","[((energy and extractives, transmission and di..."
3,a8cae1fa067f497581061dbe857cfd18,Abeyance Cryo Solutions A440 Ln2 Vapor Freezer,HHS-NIH-NIDA(AG)-SBSS-75N95021Q00250,"Health And Human Services, Department Of",75.0,National Institutes Of H

time: 37.7 s (started: 2021-07-25 16:13:17 +00:00)


In [28]:
rdf = rdf.rename(
    columns={
        'aac_code': 'aac_code',
        'active': 'active',
        'additional_info_link': 'additional_info_link',
        'address': 'address',
        'archive_date': 'archive_date',
        'archive_type': 'archive_type',
        'aug_id': 'aug_id',
        'award_date': 'award_date',
        'award_number': 'award_number',
        'award_usd': 'award_usd',
        'awardee': 'awardee',
        'basetype': 'basetype',
        'cgac': 'cgac',
        'city': 'city',
        'classification_code': 'classification_code',
        'country_code': 'country_code',
        'country_name': 'country_name',
        'countrycode': 'countrycode',
        'department_or_industry_agency': 'department_or_industry_agency',
        'description': 'description',
        'fpds_code': 'fpds_code',
        'geometry': 'geometry',
        'investment_amt_usd': 'investment_amt_usd',
        'latitude': 'latitude',
        'longitude': 'longitude',
        'naics_code': 'naics_code',
        'naics_name': 'industry',
        'new_sector': 'identified_sector',
        'new_subsector': 'identified_subsector',
        'office': 'office',
        'organization_type': 'organization_type',
        'pop_city': 'pop_city',
        'pop_country': 'pop_country',
        'pop_state': 'pop_state',
        'pop_street_address': 'pop_street_address',
        'pop_zip': 'pop_zip',
        'posted_date': 'posted_date',
        'primary_contact_email': 'primary_contact_email',
        'primary_contact_fax': 'primary_contact_fax',
        'primary_contact_fullname': 'primary_contact_fullname',
        'primary_contact_phone': 'primary_contact_phone',
        'primary_contact_title': 'primary_contact_title',
        'project_or_tender': 'project_or_tender',
        'region_code': 'region_code',
        'region_name': 'region_name',
        'response_deadline': 'response_deadline',
        'secondary_contact_email': 'secondary_contact_email',
        'secondary_contact_fax': 'secondary_contact_fax',
        'secondary_contact_fullname': 'secondary_contact_fullname',
        'secondary_contact_phone': 'secondary_contact_phone',
        'secondary_contact_title': 'secondary_contact_title',
        'sector_subsector_tuple': 'identified_sector_subsector_tuple',
        'set_a_side': 'set_a_side',
        'set_a_side_code': 'set_a_side_code',
        'solicitation_id': 'solicitation_id',
        'source': 'source',
        'state': 'state',
        'sub_tier': 'sub_tier',
        'tender_id': 'id',
        'tender_name': 'name',
        'type': 'stage',
        'url': 'url',
        'usable_location_string': 'usable_location_string',
        'zipcode': 'zipcode'
    }
)

rdf

,id,name,solicitation_id,department_or_industry_agency,cgac,sub_tier,fpds_code,office,aac_code,posted_date,stage,basetype,archive_type,archive_date,set_a_side_code,set_a_side,response_deadline,naics_code,classification_code,pop_street_address,pop_city,pop_state,pop_zip,pop_country,active,award_number,award_date,award_usd,awardee,primary_contact_title,primary_contact_fullname,primary_contact_email,primary_contact_phone,primary_contact_fax,secondary_contact_title,secondary_contact_fullname,secondary_contact_email,secondary_contact_phone,secondary_contact_fax,organization_type,state,city,zipcode,countrycode,additional_info_link,url,description,country_code,country_name,region_name,region_code,investment_amt_usd,industry,project_or_tender,source,aug_id,usable_location_string,latitude,longitude,geometry,address,identified_sector,identified_subsector,identified_sector_subsector_tuple
0,f46824eb239941a7ae2e32f0171b0d4e,Healthy Aging In Neighborhoods Of Diversity Ac...,HHS-NIH-NIDA(AG)-SBSS-75N95021Q00251,"Health And Human Services, Department Of",75.0,National Institutes Of Health,7529,NATIONAL INSTITUTES OF HEALTH NIDA,75N950,2021-06-22,Sources Sought,Sources Sought,auto15,2021-07-21,NaN,NaN,2021-07-06,541611,R499,NaN,NaN,NaN,NaN,USA,Yes,NaN,NaN,NaN,NaN,NaN,Fred Ettehadieh,Fred.Ettehadieh@nih.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OFFICE,MD,Bethesda,20892,USA,NaN,https://beta.sam.gov/opp/f46824eb239941a7ae2e3...,This is a Small Business Sources Sought notice...,USA,U.S.,North America,NAC,NaN,administrative management and general manageme...,T,SAMGOVCO,SAMGOVCO_f46824eb239941a7ae2e32f0171b0d4e,"Bethesda, USA",38.981985,-77.124234,POINT (-77.12423413116392 38.9819845),"Bethesda, Montgomery County, Maryland, United ...","[health, health, industry, trade and services,...","[administration, medicine, services, infrastru...","[((industry, trade and services, services), h)..."
1,8c98677a2f8e4de48bc5759edba78f91,Home Living Essentials & Mattresses,140A2321Q0424,"Interior, Department Of The",14.0,Bureau Of Indian Affairs,1450,INDIAN EDUCATION ACQUISITION OFFICE,140A23,2021-06-22,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,auto15,2021-07-10,ISBEE,Indian Small Business Economic Enterprise (ISB...,2021-06-25,337125,7105,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,"Begay, Janice",Janice.Begay@bie.edu,7857498419,7858302760,NaN,NaN,NaN,NaN,NaN,OFFICE,NM,Albuquerque,87104,USA,NaN,https://beta.sam.gov/opp/8c98677a2f8e4de48bc57...,Quotes must be submitted to janice.begay@bie.e...,USA,U.S.,North America,NAC,NaN,household furniture (except wood and metal) ma...,T,SAMGOVCO,SAMGOVCO_8c98677a2f8e4de48bc5759edba78f91,"Albuquerque, USA",35.084103,-106.650985,POINT (-106.650985 35.0841034),"Albuquerque, Bernalillo County, New Mexico, Un...","[industry, trade and services, industry, trade...","[housing, housing, manufacturing, manufacturin...","[((industry, trade and services, housing), h),..."
2,3340d5629a3d4c6c98eb8f4d3eefd9a4,Fan Coil,N0024421Q0082,Dept Of Defense,17.0,Dept Of The Navy,1700,NAVSUP FLT LOG CTR SAN DIEGO,N00244,2021-06-22,Solicitation,Solicitation,auto15,2021-07-09,NaN,NaN,2021-06-24,333415,4120,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Telephone: 6195565519,charles.e.fletcher1@navy.mil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OFFICE,CA,San Diego,92136,USA,NaN,https://beta.sam.gov/opp/3340d5629a3d4c6c98eb8...,EXTREMELY URGENT BRAND NAME OR EQUAL ALL OR NO...,USA,U.S.,North America,NAC,NaN,air-conditioning and warm air heating equipmen...,T,SAMGOVCO,SAMGOVCO_3340d5629a3d4c6c98eb8f4d3eefd9a4,"San Diego, USA",32.717420,-117.162773,POINT (-117.1627728 32.7174202),"San Diego, San Diego County, California, Unite...","[energy and extractives, energy and extractive...","[transmission and distribution, oil and gas, t...","[((energy and extractives, transmission and di..."
3,a8cae1fa067f497581061dbe857cfd18,Abeyance Cryo Solutions A440 Ln2 Vapor Freezer,HHS-NIH-NIDA(AG)-SBSS-75N95021Q00250,"Health And Human Services, Department Of",75.0,National Insti

time: 450 ms (started: 2021-07-25 16:27:53 +00:00)


In [30]:
push_csv_to_buffer_bucket(bucket, rdf, 'projects/samgovco/standardize/year_wise/ContractOpportunitiesFullCSV.csv')

Successfully pushed the csv file to "projects/samgovco/standardize/year_wise/ContractOpportunitiesFullCSV.csv"
time: 26.9 s (started: 2021-07-25 16:29:42 +00:00)
